In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:

# 1) Set parameters
data_dir = 'data_proc' # data_loc is  the directory on local ssd (only two sessions, one for jm049 and one for jm048)
experimenter = 'jm'
mouse = 'jm064' # 'jm049' or 'jm048'
session =  '2025-11-18_calib' # '2025-05-23_b' or '2025-05-08_c'
protocol = 'power' # 'power', 'size', 'xshift' ('yshift'), 'zshift'

ctrl_idxs = [12, 13, 14] # usually: [15, 16, 17, 18, 19] indexes of control simtuations in the stim protocol

In [ ]:
data_path = os.path.join(data_dir, experimenter, mouse, session, protocol)
# now find all .npy files in data_path matching protocol
file_list = [f for f in os.listdir(data_path) if f.endswith(f'.npy')]

In [ ]:
# now get all the x values from the filenames (the integer in the filename)
x_values = []
for f in file_list:
    # go through filename and check for each character if it is a digit
    for i, c in enumerate(f):
        if c.isdigit():
            # now get the full number (could be multiple digits)
            num_str = ''
            while i < len(f) and f[i].isdigit():
                num_str += f[i]
                i += 1
            x_values.append(int(num_str))
            break

x_values = np.array(x_values)




In [ ]:
# now sort the file_list and x_values based on x_values
sorted_indices = np.argsort(x_values)
file_list = [file_list[i] for i in sorted_indices]
x_values = x_values[sorted_indices]

In [ ]:
# now load each file and aggregate them to matrix of size (num_neurons, num_conditions)
resp_matrix = []
for f in file_list:
    resp = np.load(os.path.join(data_path, f))
    resp_matrix.append(resp)
resp_matrix = np.stack(resp_matrix, axis=1)  # shape (num_neurons, num_conditions)


In [ ]:
def plot_calib(x, resp_matrix, ctrl_idxs=None, protocol='power'):

    if protocol == 'power':
        color = 'C0'
    elif protocol == 'time':
        color = 'C1'
    elif protocol == 'xshift' or protocol == 'yshift':
        color = 'C2'
    elif protocol == 'zshift':
        color = 'C3'

    plt.figure(figsize=(2,4.5), dpi=300) 
    # plot each neuron as a grey line with alpha 0.3
    # and add a mark at each sample point
    # then add mean in black with larger markers
    for i in range(resp_matrix.shape[0]):
        if ctrl_idxs is not None and i in ctrl_idxs:
            plt.plot(x, resp_matrix[i,:], color='grey', alpha=0.3, marker='o', markersize=4, label='Ctrl-' if i==ctrl_idxs[0] else "")
        else:
            plt.plot(x, resp_matrix[i,:], color=color, alpha=0.1, marker='o', markersize=4, label='ChRm+' if i==0 else "")

    # --- compute separate means ---
    
    # convert to numpy array
    ctrl = np.array(ctrl_idxs) if ctrl_idxs is not None else np.array([])

    # all other indices
    non_ctrl = np.array([i for i in range(resp_matrix.shape[0]) if i not in ctrl])

    mean_ctrl = resp_matrix[ctrl].mean(axis=0) if ctrl.size > 0 else np.zeros(resp_matrix.shape[1])
    mean_non_ctrl = resp_matrix[non_ctrl].mean(axis=0) if non_ctrl.size > 0 else np.zeros(resp_matrix.shape[1])
    
    # --- plot the means ---
    plt.plot(x, mean_non_ctrl, color=color, marker='o', markersize=8,
            linewidth=2, label='ChRm+ mn.')
            
    plt.plot(x, mean_ctrl, color='grey', marker='o', markersize=8,
            linewidth=2, label='Ctrl mn.') if ctrl.size > 0 else None


    plt.xlabel(f'{protocol}')
    plt.ylabel('dF/F')

    plt.xticks(x)


    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    ax.set_ylim([-25, 200])



    plt.legend(fontsize=7)
    plt.show()

In [ ]:
plot_calib(x_values, resp_matrix, ctrl_idxs=ctrl_idxs, protocol=protocol)